In [105]:
import httpx
import asyncio
import anyio
import os
from datetime import datetime
import json
import re

In [106]:
payloadPath = "jsonFiles\\actualFiles\\canadaOscillatorsPayload.json"

In [107]:
file = open(payloadPath).read()

In [117]:
def extractCatName(country,payloadName):

    """
    Description: Grabs the names of the tables from a fileName.
    country: a string of the name of the country in question
    
    """
    myString = payloadName

    
    matchAgainst = '{}(.+?)Payload*'.format(country)
    
    n = re.search(matchAgainst,myString)
    #print(n.group(1))
    return n.group(1)
    

In [118]:
extractCatName('canada','canadaOscillatorsPayload.json')

'Oscillators'

In [109]:
async def async_requests(client, payloadJSONs, url):
    
    resp_jsons = []
    for payload in payloadJSONs:
        with open(payload, 'r') as f:
            
            
            conn = await client.post(url, data = f.read())
            resp_jsons.append(conn.json())
        
    return resp_jsons


In [132]:
def getJsonFromDir(directory):
    
    payloadJSONs = []
    
    files_list = os.listdir(directory)
    
    for file in files_list:
        
        with open(directory + file, 'r') as f:
            jstr = f.read()
            
            payloadJSONs.append([extractCatName('canada', file), json.loads(jstr)])
            
            
    
    return payloadJSONs

In [133]:
getJsonFromDir('E:\\webscr\\tradingView\\jsonFiles\\ActualFiles\\')

[['BalanceSheet',
  {'columns': ['description',
    'total_assets_fq',
    'fundamental_currency_code',
    'total_current_assets_fq',
    'cash_n_short_term_invest_fq',
    'total_liabilities_fq',
    'total_debt_fq',
    'net_debt_fq',
    'total_equity_fq',
    'current_ratio_fq',
    'quick_ratio_fq',
    'debt_to_equity_fq',
    'cash_n_short_term_invest_to_total_debt_fq'],
   'ignore_unknown_fields': False,
   'options': {'lang': 'en'},
   'range': [0, 100],
   'sort': {'sortBy': 'market_cap_basic',
    'sortOrder': 'desc',
    'nullsFirst': False},
   'preset': 'large_cap'}],
 ['CashFlow',
  {'columns': ['description',
    'cash_f_operating_activities_ttm',
    'fundamental_currency_code',
    'cash_f_investing_activities_ttm',
    'cash_f_financing_activities_ttm',
    'free_cash_flow_ttm',
    'capital_expenditures_ttm'],
   'ignore_unknown_fields': False,
   'options': {'lang': 'en'},
   'range': [0, 100],
   'sort': {'sortBy': 'market_cap_basic',
    'sortOrder': 'desc',
   

In [78]:
async with httpx.AsyncClient() as client:
    

    resps_list = await async_requests(client, requests_list, "https://scanner.tradingview.com//egypt//scan")
    


<Response [200 OK]>
<Response [200 OK]>
<Response [200 OK]>
<Response [200 OK]>
<Response [200 OK]>
<Response [200 OK]>
<Response [200 OK]>
<Response [200 OK]>
0:00:02.432907


In [79]:
def countryAllCSVS(payloadsPath, country):
    
    tables = []
    tablesCap = []
    payloads = os.listdir(payloadsPath)
    
    
    for payload in payloads:
        tableName = extractCatName('canada',payload)
        tablesCap.append(tableName)
        tables.append(tableName.lower())
    
    
    for table in tables:

        for dic in dictsList:
            
            if dic['title'] == table:
                
                #print("{} == {}".format(dic['title'],table))
                payloadString = basePath + "jsonFiles\\ActualFiles\\canada{}Payload.json".format(tablesCap[tables.index(table)])
                resp = getCountryTable((payloadString), country, reqRows = 20000)
                createCsv(country+table.capitalize(), country[1], resp, dic, json.loads(open(basePath + 'jsonFiles\\ActualFiles\\canada{}Payload.json'.format(tablesCap[tables.index(table)])).read()))
   
    

In [80]:
def createCsv(fileName, country, response, valuesDict, payloadJSON):
    
    """
    Response: sever's response (JSON)
    fileName: Name of file you'll be creating to write the result to
    Columns: A list that will be written to the first row of the csv
    
    """
    
    columns = craftColumns(valuesDict, payloadJSON)
    if country not in os.listdir(basePath + 'csvs\\'):
        os.mkdir(basePath + 'csvs\\{}'.format(country))

    with open(basePath + 'csvs\\{}\\{}.csv'.format(country, fileName), 'w', newline='') as file:

        
        serverResp = response['data']
        writer = csv.writer(file)
        writer.writerow(columns)

        for i in range(len(serverResp)):

            writer.writerow(serverResp[i]['d'])

In [81]:
def craftColumns(myDict, payloadJSON):
    
    """
    myDict: A hardcoded dicitionary that holds payload values and their corresponding manifistation on the page 
    myJSON: a json of the values sent in the payload
    Returns: A list of the column names pulled from the page itself as opposed to keys used to address the API
    """
    
    pageVals = []
    myColumns = payloadJSON['columns']
    
    for row in myColumns:
        
        if row in myDict.values():

            for key,val in myDict.items():
                

                if val == row:
                    pageVals.append(key)

        else:
            pageVals.append(row)

    return pageVals

In [44]:
reli = os.listdir("jsonFiles\\actualFiles")

In [45]:
requests_list = ["jsonFiles\\actualFiles\\" + rel for rel in reli]

In [60]:
with open(requests_list[0], 'r') as f:
    print(f.read())
    

{"columns":["description","total_assets_fq","fundamental_currency_code","total_current_assets_fq","cash_n_short_term_invest_fq","total_liabilities_fq","total_debt_fq","net_debt_fq","total_equity_fq","current_ratio_fq","quick_ratio_fq","debt_to_equity_fq","cash_n_short_term_invest_to_total_debt_fq"],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,100],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc","nullsFirst":false},"preset":"large_cap"}


In [46]:
requests_list

['jsonFiles\\actualFiles\\canadaBalanceSheetPayload.json',
 'jsonFiles\\actualFiles\\canadaCashFlowPayload.json',
 'jsonFiles\\actualFiles\\canadaIncomeStatementPayload.json',
 'jsonFiles\\actualFiles\\canadaOscillatorsPayload.json',
 'jsonFiles\\actualFiles\\canadaPerformancePayload.json',
 'jsonFiles\\actualFiles\\canadaTrendFollowingPayload.json',
 'jsonFiles\\actualFiles\\canadaTrendFollowPayload.json',
 'jsonFiles\\actualFiles\\canadaValuationPayload.json']